In [1]:
import pandas as pd

from src.pipelines import pipeline_data_collection, pipeline_data_processing

pd.set_option('display.max_columns', 50)

# Data collection

In [2]:
update_tables = dict(despesas=True, 
                     deputados=False)
id_deputados_to_fetch = [220526, 178873, 220527, 220530, 160517, 
                         220528, 204356, 204355, 178871, 220529]
start_date = '2024-05-01'
end_date = '2024-05-01'

pipeline_data_collection(update_tables, id_deputados_to_fetch, start_date, end_date)

ID:220526, 2024-05: 20 registros
ID:178873, 2024-05: 6 registros
ID:220527, 2024-05: 2 registros
ID:220530, 2024-05: 6 registros
ID:160517, 2024-05: 4 registros
ID:220528, 2024-05: 11 registros
ID:204356, 2024-05: 0 registros
ID:204355, 2024-05: 17 registros
ID:178871, 2024-05: 1 registros
ID:220529, 2024-05: 6 registros


In [3]:
update_tables = dict(despesas=True, 
                     deputados=True)

pipeline_data_processing(update_tables)

In [ ]:
deputados = pd.read_parquet('data/cleansed/deputados.parquet')
print(deputados.shape)

despesas = pd.read_parquet('data/cleansed/despesas.parquet')
print(despesas.shape)

df = pd.merge(despesas, deputados, how='left', 
              left_on='id_deputado', right_index=True)
print(df.shape)

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
tmp = df.groupby(['id_deputado', 'ano_mes'])[['valorDocumento']].sum()
tmp.reset_index(inplace=True)
tmp = pd.merge(tmp, deputados, how='left', left_on='id_deputado', right_index=True)

In [ ]:
px.bar(df.loc[df['ano_mes']=='2024_04'], x='nome', y='valorDocumento', color='tipoDespesa')

In [ ]:
df.groupby('nomeFornecedor')['valorDocumento'].sum().sort_values(ascending=False).head(10)

In [ ]:
df.loc[(df['nome'].str.contains('Linhalis')) & (df['ano_mes']=='2023_12') & (df['valorDocumento'].between(10000, 13000)), ['urlDocumento']].values

In [ ]:
px.line(tmp, x='ano_mes', y='valorDocumento', color='nome')

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
tmp = df.groupby(['id_deputado', 'ano_mes'])[['valorDocumento']].sum()

fig = go.Figure()
for k, tmpdf in df.groupby(['id_deputado']):
    px.line(tmp.loc[dep], y='valorDocumento')
    # fig.add_trace(px.line(tmp.loc[dep], y='valorDocumento'))
fig.show()

# Rascunho

In [ ]:
def pipeline_data_collection():
    info_datasets = {} # config
    reprocessing_status = {}
    for dataset in info_datasets.keys():
        df = collect_data(dataset)
        reprocessing_status[dataset] = get_reprocessing_status()
        if reprocessing_status[dataset]:
            save_data(df)

    save_data(reprocessing_status)

In [ ]:
def pipeline_data_processing():
    
    process_deputados()
    process_despesas()

In [ ]:
pipeline_data_collection()

pipeline_data_processing()

update_website()